# Abstract: Pose与Structure IO

蛋白质是大型生物分子，它由一个或多个由α-氨基酸残基组成的长链条组成。α-氨基酸分子呈线性排列，相邻α-氨基酸残基的羧基和氨基通过肽键连接在一起。
蛋白质的分子结构可划分为四级，以描述其不同的方面：
- 蛋白质一级结构：组成蛋白质多肽链的线性氨基酸序列。
- 蛋白质二级结构：依靠不同氨基酸之间的C=O和N-H基团间的氢键形成的稳定结构，主要为α螺旋和β折叠。
- 蛋白质三级结构：通过多个二级结构元素在三维空间的排列所形成的一个蛋白质分子的三维结构。
- 蛋白质四级结构：用于描述由不同多肽链（亚基）间相互作用形成具有功能的蛋白质复合物分子。

![title](./img/pose.png)

在这个章节中，你即将会学习到以下的一些信息：
- Pose & Structure IO
- PDBinfo & Pose
- Atom、Residue & ResidueType
- Conformation & Protein Geometry
- Pose operating

### 简介: Pose的组织构架
因此如果要在计算机中建立一个蛋白质的结构模型，就清楚地描述每一个原子的信息。在Rosetta中，Pose是管理蛋白质信息的中心，可以描述蛋白质一到四级结构所有的信息。而且这些信息是分层管理的比如:

- Conformation: 负责管理原子类型(AtomType)、氨基酸类型(ResidueType)、氨基酸的原子坐标(xyz)、氨基酸连接方式的定义(FoldTree/AtomTree)等，这部分构成了蛋白质构象的所有物理信息。(最重要)
- Energy: 负责管理氨基酸直接的能量计算所需的信息(EnergyGraph/energies)
- ConstraintSet: 负责管理原子间的约束信息(constraints)
- DataCache: 负责管理用户自定义的信息

分层式管理使得Pose的信息修改和更新变得容易。

除此以外，还有些外部对象如PDBinfo也负责转换和储存Pose与PDB之间的信息变换。

以下是一个Pose中的架构的示意图：
![title](./img/PoseObject.png)

### 1.4 Pose中的结构信息
PDB的信息可以通过PDBinfo获取，除此以外，Pose中还有大量的结构信息，我们可以轻松通过各类函数来获取更多的信息，以下将逐一地列举。

#### 1.4.1 一级与二级结构信息的提取

In [ ]:
# Pose中的基本信息，如残基数量，序列，FoldTree(后续讲述)
print(pose)

**Tips: 除此以外，还有一些具体的方法可以获取更多的细节:**

In [ ]:
# 返回pose中链的数目
pose.num_chains()

In [ ]:
# 获取Pose的序列信息的方法
print(f'所有的氨基酸:{pose.sequence()}\n')
print(f'前5个氨基酸:{pose.sequence(1,5)}\n')
print(f'1号链的所有氨基酸:{pose.chain_sequence(1)}\n')
print(f'2号链的所有氨基酸:{pose.chain_sequence(2)}\n')

In [ ]:
# 获取Pose的氨基酸总数量方法
pose.total_residue()

In [ ]:
# 通过DSSP获取二级结构信息
from pyrosetta.rosetta.protocols.membrane import get_secstruct
get_secstruct(pose)

#### 1.4.2 氨基酸信息
Residue Object是Pose的重要组成部分，整个Pose的Conformation是由一个个具体的氨基酸的具体构象组成，每个氨基酸有一个单独的object来描述，其中记录了所有的氨基酸信息。
同样，通过Pose类，我们可以轻松地访问每个氨基酸对象，并提取其中的信息。

##### 结果解读:
使用print打印信息后，可以获取这个对象中所有的信息:</br>
如: 氨基酸的类型为丝氨酸(SER), 骨架原子和侧链的原子组成、残基性质、以及每个单独原子的三维坐标信息

#### 1.4.4 蛋白质几何信息
Pose中描述具体的蛋白质构象，除了氨基酸类型以外，更是由原子间的键长、键角，二面角等一系列的具体参数构成。Pose中的Conformation对象负责记录了这些连接信息。
![title](./img/phipsiomega.png)

为了定位原子的信息，首先需要构建atom identifier对象，相当于创建一个ID卡，让Rosetta知道我们指定的原子是位于哪个氨基酸中的。通过AtomID，提供残基号，原子号，就可以创建atom identifier对象

In [ ]:
# 获取原子间的键长、键角信息前需要构建atom identifier objects
from pyrosetta.rosetta.core.id import AtomID
atom1 = AtomID(1, 24)  # 24号残基的第一个原子
atom2 = AtomID(2, 24)  # 24号残基的第二个原子
atom3 = AtomID(3, 24)  # 24号残基的第三个原子
atom4 = AtomID(4, 24)  # 24号残基的第四个原子
print(atom1, atom2, atom3, atom4)

知道原子的ID后，就可以轻松的通过conformation对象来获取键长、键角等数据了。一般在Rosetta中键长和键角都设定为**理想值**。

In [ ]:
# 通过conformation层获取键长数据
bond_length = pose.conformation().bond_length(atom1, atom2)

# 通过conformation层获取键角数据(弧度)
bond_angle = pose.conformation().bond_angle(atom1, atom2, atom3)

print(f'原始键长:{bond_length}, 原始键角:{bond_angle}')

In [ ]:
# 通过pose获取氨基酸的骨架二面角数据
phi = pose.phi(24)
psi = pose.psi(24)
omega = pose.omega(24)
print(f'原始phi角:{phi}, 原始psi角:{psi}, 原始omega角:{omega}')

### 1.5 Pose的操作
Pose中储存了非常多的信息，同时还提供了接口可以让用户方便的对其中的信息进行修改（采样）。

#### 1.5.1 Pose的创建和复制
前几节中提及过，pose是一个容器，理所当然我们可以创建一个空的容器，里面什么都不放。</br>
很多时候，我们需要创建空的Pose对象，便于保存当前pose的实例化状态，可作为可回溯点或初始状态，方便反复调用。

In [ ]:
# 通过创建一个新的Pose
from pyrosetta import Pose
starting_pose = Pose()
starting_pose2 = None

此处通过两种方法，将已有的Pose信息放入新的容器里，一种是通过assign函数复制，一种是通过python赋值符号来赋值。

In [ ]:
# 方法1：通过assign复制新的构象
starting_pose.assign(pose)

# 方法2：通过python的直接赋值符号
starting_pose2 = pose

print(starting_pose)
print('\n')
print(starting_pose2)

**结果解读:</br>
可见，两种方式“看”起来里面都有了新的pose信息。但真的如此么？**

In [ ]:
# 尝试调整mypose中的24号氨基酸phi值:
pose.set_phi(24, 170.0)

# 查看对starting_pose以及starting_pose2的影响:
print(f'starting_pose 24 residue phi:{starting_pose.phi(24)}')
print(f'starting_pose2 24 residue phi:{starting_pose2.phi(24)}')

##### 结果解读:
结果可见，starting_pose2是用过python直接赋值的Pose并没有复制，而只是pose的一个"超链接"符，并没有进行"复制"的操作。
而通过assign的复制，原始的pose的任何调整都没有对starting_pose造成任何的影响，可见其独立性。

#### 1.5.2 链与氨基酸的增减操作
除了对整体信息的迁移，用户还可以对Pose中的链以及氨基酸进行操作。

##### **1.5.2.1 链的切割处理**

尽管pose的氨基酸编号是忽略多肽链的分隔的，但是pose中的链依然是根据多肽链的物理结构进行编号的，同理也是从1开始编号。</br>
如一个蛋白中有2条链A和B，那么链编号结果即为1和2。其中A对应1号链，B对应2号链，与PDB的链顺序有关（当然A链的顺序如果在后面，那么B链就是1号链）。</br>
Pose类中许多的方法可以很方便对链的增减进行操作, 以下2个举例进行说明:

In [ ]:
# 将Pose按照链的数量进行切割
pose_list = pose.split_by_chain()
pose_list

此处的pose_list中存放了2个数据，说明链已经被切割成2个独立的pose对象了。</br>
通过python的索引，可以获得具体的pose:

In [ ]:
# 获取只含有第一个链的pose
chain1_pose = pose.split_by_chain()[1]  # 直接切片索引链号。
chain2_pose = pose.split_by_chain()[2]  # 直接切片索引链号。

# check
print(f'chain1_pose中的氨基酸总数:{chain1_pose.total_residue()}')
print(f'chain2_pose中的氨基酸总数:{chain2_pose.total_residue()}')
print(f'原始pose中的氨基酸总数:{pose.total_residue()}')

##### **1.5.2.2 链的合并处理**

除了分隔操作，用户还可以通过一些简单的方式把链合并到一个pose中，此处使用append_pose_to_pose函数就可以达到目的。但需要注意，pose中的氨基酸、链的数量变化后，都需要对PDBinfo进行更新。否则PDBinfo的信息与Pose信息不对称。

In [ ]:
# 两条链的合并;
# add binder to pose;
from pyrosetta.rosetta.core.pose import append_pose_to_pose
print(f'原始chain1_pose中的氨基酸总数:{chain1_pose.total_residue()}')
append_pose_to_pose(chain1_pose, chain2_pose)
chain1_pose.update_residue_neighbors()
chain1_pose.update_pose_chains_from_pdb_chains()
chain1_pose.conformation().detect_disulfides()

# update pdbinfo; 别忘了更新pdbinfo;
# 更新pdb_info; [别忘了]
from pyrosetta.rosetta.core.pose import renumber_pdbinfo_based_on_conf_chains
renumber_pdbinfo_based_on_conf_chains(pose)

print(f'append之后的chain1_pose中的氨基酸总数:{chain1_pose.total_residue()}')
chain1_pose.sequence()

# 检查PDBinfo是否正确: Returns true if PDBInfo is obsolete and needs updating
print(f'PDBinfo是否需要被更新:{pose.pdb_info().obsolete()}')

##### **1.5.3.3 氨基酸的删减操作**
除了对链的合并之外，我们还可以对链中的氨基酸进行添加、删除的操作！具体的过程是用户需要创建一个独立的氨基酸(residue object)，并将这个氨基酸加载到现有的构像中。</br>
加载的方式可以是前置后后置，根据使用的函数不同而定。

In [ ]:
# 在链的前端添加新的氨基酸或删除氨基酸
from pyrosetta.rosetta.core.conformation import ResidueFactory
from pyrosetta.rosetta.core.chemical import ChemicalManager

print(f'原始氨基酸总数:{pose.total_residue()}')
print(f'原始氨基酸序列:{pose.sequence()}\n')

# 向前添加氨基酸
chm = ChemicalManager.get_instance()
rts = chm.residue_type_set("fa_standard")
new_rsd = ResidueFactory.create_residue(rts.name_map('ALA')) # 创建一个residue object
pose.prepend_polymer_residue_before_seqpos(new_rsd, 1, True)  # 在第一个氨基酸前添加一个ALA

print(f'向前添加之后氨基酸总数:{pose.total_residue()}')
print(f'向前添加之后氨基酸序列:{pose.sequence()}\n')

In [ ]:
# 向后添加氨基酸
last_residue = pose.total_residue()
pose.append_polymer_residue_after_seqpos(new_rsd, last_residue, True)  # 在第一个氨基酸前添加一个ALA

print(f'向后添加之后氨基酸总数:{pose.total_residue()}')
print(f'向后添加之后氨基酸序列:{pose.sequence()}\n')

In [ ]:
# 删除氨基酸
pose.delete_polymer_residue(1)  # 删除第一个氨基酸

print(f'删除之后氨基酸总数:{pose.total_residue()}')
print(f'删除之后氨基酸序列:{pose.sequence()}\n')

In [ ]:
# 还可以范围性的删除氨基酸
pose.delete_residue_range_slow(1,5) # 删除第一个至第五个氨基酸

print(f'删除之后氨基酸总数:{pose.total_residue()}')
print(f'删除之后氨基酸序列:{pose.sequence()}\n')

##### **1.5.3.4 PBDinfo更新**

In [ ]:
# 更新pdb_info; [别忘了!]
from pyrosetta.rosetta.core.pose import renumber_pdbinfo_based_on_conf_chains

renumber_pdbinfo_based_on_conf_chains(pose)  # 更新PDBinfo.

# 检查PDBinfo是否正确: Returns true if PDBInfo is obsolete and needs updating
print(f'PDBinfo是否需要被更新:{pose.pdb_info().obsolete()}')

#### 1.5.3 构象的调整

除了对多肽链的氨基酸数量的调整，我们还可以通过Pose中的一些函数来调整蛋白质的具体构象，如主链的phi/psi角、化学键中的键长与键角数据等。

##### **1.5.3.1 化学键的数据调整**

In [ ]:
# 修改键长键角必须通过conformation层进行处理:
print(f'原始键长:{bond_angle}, 原始键角:{bond_length}')

pose.conformation().set_bond_angle(atom1, atom2, atom3, 0.66666 * 3.14)
new_bond_angle = pose.conformation().bond_angle(atom1, atom2, atom3)

pose.conformation().set_bond_length(atom1, atom2, 1.44)
new_bond_length = pose.conformation().bond_length(atom1, atom2)

print(f'新的键长:{new_bond_length}, 新的键角:{new_bond_angle}')

In [ ]:
# 修改phi、psi、chi、omega角可以直接通过pose的函数:
# 通过pose获取氨基酸的骨架二面角数据
print(f'原始phi角:{pose.phi(24)}, 原始psi角:{pose.psi(24)}, 原始omega角:{pose.omega(24)}')
pose.set_phi(24, 66.0)
pose.set_psi(24, 55.0)
pose.set_omega(24, 180.0)

print(f'调整后phi角:{pose.phi(24)}, 调整后psi角:{pose.psi(24)}, 调整后omega角:{pose.omega(24)}')

##### **1.5.3.2 氨基酸类型的调整(突变)**
除了具体的化学键数据的调整，在PyRosetta中进行氨基酸的类型调整也是很方便的

In [ ]:
# 调整氨基酸的类型
from pyrosetta.toolbox import mutate_residue
print(f'原始氨基酸类型:{pose.residue(1).name()}')
print('突变氨基酸中...')
mutate_residue(pose, 1, 'A', 9.0)  # 1 代表氨基酸突变的pose编号，9.0代表对氨基酸附近9埃范围内的氨基酸进行侧链优化，适应新的突变。
print(f'突变后氨基酸类型:{pose.residue(1).name()}')

##### **1.5.3.3 原子坐标的修改**
原子坐标的修改需要获取residue对象，并获取原子ID(atom identifier objects)。通过pose.set_xyz函数设定新的xyz坐标, 但用户一般不需要”显式“地修改原子坐标, 除非你明白这样操作的意义。</br>
此处以创建一个镜像原子进行说明:

In [ ]:
# 原子坐标的修改（一般不需要这样操作）
from pyrosetta.rosetta.numeric import xyzVector_double_t

# 对第24个氨基酸的所有原子的x坐标乘上一个负号:
residue24 = pose.residue(24)  # 获取residue对象
for atom_id, atom in enumerate(residue24.atoms()):
    x, y, z = atom.xyz()
    print(f'坐标进行修改前信息: 原子号:{atom_id+1}, x:{x}, y:{y}, z:{z}')
    
    mirror_xyz = xyzVector_double_t(-x, y, z)  # 乘上负号.
    atom_index = AtomID(atom_id+1, 24)   # 24号氨基酸的第x个原子的id
    pose.set_xyz(atom_index, mirror_xyz) # 设置xyz坐标

print('\n')
    
for atom_id, atom in enumerate(residue24.atoms()):
    x, y, z = atom.xyz()
    print(f'坐标进行修改后信息:  原子号:{atom_id+1}, x:{x}, y:{y}, z:{z}')

### 1.6 Pose的能量
如果现在我们已有一个Pose，我们想评估这个蛋白质的能量评分，可以直接通过创建一个打分函数并对Pose的能量进行计算，</br>
再可以通过pose中的energies对象将氨基酸残基的One-body, Two-body的能量信息列出，达到残基级别能量"分解"的目的。

#### 1.6.1 对结构进行能量计算
create_score_function函数可以用于快速创建一个打分函数。

In [ ]:
## 创建标准打分函数
from pyrosetta import create_score_function
scorefxn = create_score_function('ref2015')

# 对当前Pose中的构象进行能量计算
weighted_total_score = scorefxn(pose)
print(weighted_total_score)

#### 1.6.2 能量信息
Rosetta的能量是加权后的能量，每个能量项有自己的权重，通过energies获取的是能量项的原始结果（unweighted）。

In [ ]:
# 获取能量对象
scores = pose.energies()

# 获取1号残基的所有能量项的信息:
print(scores.show(1))

Rosetta中的Score项有许多，如fa_atr代表范德华吸引势力, fa_rep代表范德华排斥项, fa_elec代表静电项等。</br>
比如通过ScoreType类下的属性，即可搜索到对应的能量项。</br>
获取总能中的某一个项的值可以直接使用python的索引功能，十分方便:

In [ ]:
# 获取总fa_atr项能量项的得分结果:
from pyrosetta.rosetta.core.scoring import ScoreType
pose.energies().total_energies()[ScoreType.fa_atr]

In [ ]:
# 单独获得第5号氨基酸的fa_atr项能量得分:
pose.energies().residue_total_energies(5)[ScoreType.fa_atr]

### 1.7 自定义信息
Pose中含有让用户自定义写入任何信息的功能，比如在程序设计过程中，中间生成的临时数值或字符都可以写入到PoseExtraScore中，这些信息会随着Pose一并输出到PDB或则Silent文件中，在后续的分析和处理的过程中非常方便。

In [ ]:
# 给pose加入额外的信息: 比如filter计算的值就可以储存.
from pyrosetta.rosetta.core.pose import setPoseExtraScore, getPoseExtraScore

setPoseExtraScore(pose, "distance", 1.0)
setPoseExtraScore(pose, "angle", 120.5)

# 提取信息
print(getPoseExtraScore(pose, 'distance'))
print(getPoseExtraScore(pose, 'angle'))  # 目前有bug，但是信息已经储存在pose中了